In [ ]:
from tqdm import tqdm
from array_lib import *
from point3d_lib import Point
from ply_creation_lib import create_ply, create_ply_normals
from skeleton3d_lib import Skeleton
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
import pydicom as dicom
import numpy as np
import itertools
import struct
import pickle
import time
import os

In [ ]:
"""
input_folder - must contain dcm files directly within it and only of one scan.
offsets - range of colors to include
seed - the entry coordinate to the aorta, must be the center.
threshold - normalized image color lower threshold
image_center - point between the arteries of interest
"""
input_folder = f'{os.getcwd()}\\20240923'
offsets = (-15, 20)
seed = (20, 310, 160)
image_center = (150, 280, 230)
min_skeleton_length = 100

In [ ]:
def read_dicom(input_folder: str) -> np.ndarray:
    files: list[str] = os.listdir(input_folder)
    data = [dicom.dcmread(f'{input_folder}\\{file}') for file in files if file.endswith('.dcm')]
    image = np.array([dicom.pixel_array(datum) for datum in data])
    return image

def normalize_image_colors(image: np.ndarray) -> np.ndarray:
    min_val = np.min(image)
    max_val = np.max(image)
    image = (image - min_val) / (max_val - min_val) * 255
    return image

image = normalize_image_colors(read_dicom(input_folder))

In [ ]:
filtered_mask = custom_floodfill_3d(image, seed_point=seed, new_value=-1, offsets=offsets)
eroded_mask = erode_3d(filtered_mask)
heartless_mask = remove_heart(eroded_mask)
trimmed_mask = distinguish_3d(heartless_mask)

In [ ]:
skeleton_mask = skeletonize(trimmed_mask)
closest_skeletons = find_closest_skeletons(skeleton_mask, image_center)
skeleton_points = np.concatenate([i[0] for i in closest_skeletons])
filtered_skeleton_mask = np.zeros_like(skeleton_mask).astype(bool)
filtered_skeleton_mask[skeleton_points[:, 0], skeleton_points[:, 1], skeleton_points[:, 2]] = True

selected_skeletons = floodfill_nearby_skeletons(heartless_mask, closest_skeletons)

In [ ]:
skeletons = [s[0] for s in closest_skeletons]

left_skel = Skeleton('left')
left_skel.add_points(skeletons[0], filtered_skeleton_mask)

right_skel = Skeleton('right')
right_skel.add_points(skeletons[1], filtered_skeleton_mask)

In [ ]:
left_skel.remove_close_ends()
right_skel.remove_close_ends()

In [ ]:
center_point = Point(image_center)
left_skel.find_head_point(center_point)
right_skel.find_head_point(center_point)

In [ ]:
branches: list[Skeleton] = []
for skel in [left_skel, right_skel]:
    branches.extend(skel.create_new_path_skeletons(min_skeleton_length))

In [ ]:
for i, branch in enumerate(branches):
    branch.calculate_normals()
    create_ply_normals(branch, f'normals_{i+1}.ply')

In [ ]:
def display_branches(branches: list[Skeleton]) -> None:
    for branch in branches:
        branch_mask = np.zeros_like(image).astype(bool)
        for point in branch.points:
            branch_mask[tuple(point.coordinates)] = True
        create_ply(branch_mask, f'{branch.name}_branch_{branch.id}.ply')
        
display_branches(branches)

In [ ]:
def display_mpr(branch: Skeleton):
    mpr_test_x = np.zeros((image.shape[0], len(branch)))
    mpr_test_y = np.zeros((image.shape[1], len(branch)))
    mpr_test_z = np.zeros((image.shape[2], len(branch)))

    for i, p in enumerate(branch.points):
        coords = p.coordinates
        x_pixels = image[:, coords[1], coords[2]]
        y_pixels = image[coords[0], :, coords[2]]
        z_pixels = image[coords[0], coords[1], :]
        
        mpr_test_x[:, i] = x_pixels
        mpr_test_y[:, i] = y_pixels
        mpr_test_z[:, i] = z_pixels

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(mpr_test_x, cmap='gray')
    axes[1].imshow(mpr_test_y, cmap='gray')
    axes[2].imshow(mpr_test_z, cmap='gray')
    fig.suptitle(branch.name)
    plt.show()
    
for branch in branches:
    display_mpr(branch)

In [ ]:
create_ply(image, f'1.0_image.ply')
create_ply(filtered_mask, f'1.1_filtered.ply')
create_ply(eroded_mask, f'1.2_eroded.ply')
create_ply(heartless_mask, f'1.3_heartless.ply')
create_ply(trimmed_mask, f'1.4_trimmed.ply')
create_ply(skeleton_mask, f'1.5_skeleton.ply')
create_ply(filtered_skeleton_mask, f'1.6_closest_skeletons.ply')